# Failure Analysis of Logistic Regression (TF-IDF + NMF + Meta)

### Goal

- Analyze failure cases (False Positives, False Negatives) of the calibrated Logistic Regression classifier to understand error patterns and improve future iterations.

### Background

- Model: TF-IDF + NMF + simple metadata (log token length, has_url, anxiety_score) → Logistic Regression with isotonic calibration.

### Data and Artifacts

- Data: `data/processed/reddit_anxiety_v1.parquet` with combined labels from `sample_human_labels.csv` (hand) and `simple_ai_labels.csv` (ai).
- Artifacts: `artifacts/vec_final.joblib`, `artifacts/nmf_final.joblib`, and `artifacts/triggerlens_logreg_calibrated_bundle.joblib` (includes calibrated model, meta scaler, and threshold).

### Method

1. Build labels: hand (anxiety_rating ≥ 4) and ai (category contains anx/panic, severity ≥ 4, confidence ≥ 0.5), then combine (prefer hand).
2. Tokenize → TF-IDF → NMF topics; add meta features; apply saved scaler.
3. Predict with calibrated LR using saved threshold.
4. Identify top FP (highest probability among negatives) and top FN (lowest probability among positives); list token-level contributions from TF-IDF weights (when available).

### Sanity Check of This Run

- Supervised samples: 1,006
- Feature shape: (1006, 9717)
- Positive rate: 173 / 1006 ≈ 17.2%
- Threshold (bundle): ~0.25
- Failures: False Positives = 120, False Negatives = 16
  These are consistent with the reference script outputs, indicating correct loading and feature construction.

### Key Results

- Operating point uses threshold ≈ 0.25 (from calibration in training notebook).
- FP = 120, FN = 16 on the combined labeled subset (N=1,006).
- The notebook prints excerpts and token contributions for the highest-confidence FP and the highest-confidence FN.

### Failure Analysis Highlights

- Highest-confidence FP: Text strongly emotive/long-form ranting; top contributing tokens include generic stress/struggle terms (e.g., "hard", "afford").
  - Observation: Lexical cues tied to hardship and apology may push probability high even in non-anxiety labels, suggesting context/sentiment overlap.
- Highest-confidence FN: Long meta-thread content with political/religion context; high-probability positive cue n-grams present but counterweighted by strong non-anxiety tokens (e.g., "religion", "god").
  - Observation: Topic/context tokens can dominate and suppress anxiety cues for general-discussion or meta posts.

### Interpreting FP/FN Patterns

- FP drivers:
  - General distress-related lexicon (apologies, hardship, finance) without explicit anxiety context.
  - Longer posts with many neutral n-grams where a subset of weighted tokens dominates.
- FN drivers:
  - Topic tokens (religion/politics) that the model has learned as non-anxiety; they can override sparse anxiety indicators.
  - Posts with diffuse, non-self-referential language or multi-topic content.

### Actionable Ideas

- Features: Add character n-grams, adjust min_df/max_features; consider sublinear TF; try class-conditional n-grams.
- Reweighting: Explore focal loss or alternative calibration/thresholds per content-type (e.g., meta threads).
- Data: Curate more positives with subtle/implicit anxiety; add hard negatives (politics/religion discussions) to disentangle topic vs. state.
- Modeling: Try linear+nonlinear hybrids (e.g., TF-IDF LR + small neural re-ranker) or stronger topic disentanglement.

### Limitations

- Combined labels have noise and potential labeling bias; moderate dataset size.
- Linear classifier may underfit complex semantics; NMF topics can drift.
- Current failure view is a single sample per FP/FN bucket; broader slices (e.g., top-k, per-subreddit) would provide more robust insights.

### Conclusion

- At the calibration-derived threshold (~0.25), recall is prioritized, yielding relatively few FNs (16) at the cost of more FPs (120).
- FP patterns indicate hardship/sentiment overlap; FN patterns indicate topic overshadowing anxiety cues.
- The findings support targeted feature and data improvements to reduce FPs without inflating FNs.


In [1]:
# STEP 0: Setup and Imports
import sys
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import joblib

from scipy.sparse import csr_matrix, hstack

# Compatibility shim for numpy/joblib artifacts
if not hasattr(np, "_core"):
    import numpy.core as core

    sys.modules["numpy._core"] = core
    sys.modules["numpy._core._multiarray_umath"] = core._multiarray_umath


# Reusable identity for joblib pipelines
def identity(x):
    return x


# Paths
NB_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
ART = NB_DIR.parent / "artifacts"
PROC = NB_DIR.parent / "data" / "processed"

TEXT_COL = "text_all"
PUNCT = ".,!?:;()[]{}\"'" "''-–—/\\"
TRASH = {"[text]", "[image]", "[removed]", "[deleted]"}
KEEP_SHORT = {"ecg", "sad", "ptsd", "mom", "dad", "anx"}


def tokenize(s: str):
    tokens = []
    for word in str(s).split():
        w = word.strip().strip(PUNCT).lower()
        if w and w not in TRASH and (len(w) >= 3 or w in KEEP_SHORT):
            tokens.append(w)
    return tokens


print("(✓) Setup complete")
print(f"  - Data path: {PROC}")
print(f"  - Artifacts path: {ART}")

(✓) Setup complete
  - Data path: /Users/mariamckay/code/umich/milestone2/data/processed
  - Artifacts path: /Users/mariamckay/code/umich/milestone2/artifacts


In [2]:
# STEP 1: Load models/artifacts
print("[1/6] Loading models...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    vec = joblib.load(ART / "vec_final.joblib")

if not hasattr(vec, "idf_") or getattr(vec, "idf_", None) is None:
    print("[ERROR] Vectorizer is not fitted properly! Please re-train models.")
else:
    print(f"  - Vectorizer loaded: {vec.__class__.__name__}")
    print(
        f"  - Vocabulary size: {len(vec.vocabulary_) if hasattr(vec, 'vocabulary_') else 'unknown'}"
    )

nmf = joblib.load(ART / "nmf_final.joblib")
bundle = joblib.load(ART / "triggerlens_logreg_calibrated_bundle.joblib")
calib_model = bundle["calibrated_model"]
meta_scaler = bundle["meta_scaler"]
threshold = float(bundle.get("threshold", 0.257))
print("  - NMF and calibrated LR loaded")

[1/6] Loading models...
  - Vectorizer loaded: TfidfVectorizer
  - Vocabulary size: 9699
  - NMF and calibrated LR loaded


In [3]:
# STEP 2: Load data
print("[2/6] Loading data...")
df_main = pd.read_parquet(PROC / "reddit_anxiety_v1.parquet")
df_hand = pd.read_csv(PROC / "sample_human_labels.csv")
df_ai = pd.read_csv(PROC / "simple_ai_labels.csv")
print(f"  - Main: {len(df_main):,} rows | Hand: {len(df_hand):,} | AI: {len(df_ai):,}")

[2/6] Loading data...
  - Main: 6,283 rows | Hand: 599 | AI: 1,000


In [4]:
# STEP 3: Build labels (hand, ai, combined)
print("[3/6] Creating labels...")
# hand
df_h = df_hand[["post_id"]].copy()
rating = pd.to_numeric(df_hand.get("anxiety_rating", pd.Series()), errors="coerce")
df_h["label"] = (rating >= 4).astype(int)
# ai
cat = df_ai["ai_category"].astype(str).str.lower()
conf = pd.to_numeric(df_ai["ai_confidence"], errors="coerce").fillna(0)
sev = pd.to_numeric(df_ai["ai_severity"], errors="coerce").fillna(0)
df_a = df_ai[["post_id"]].copy()
df_a["label"] = (
    (cat.str.contains("anx") | cat.str.contains("panic")) & (sev >= 4) & (conf >= 0.5)
).astype(int)
# combine (prefer hand)
df_h_temp = df_h.copy()
df_h_temp["source"] = "hand"
df_a_temp = df_a.copy()
df_a_temp["source"] = "ai"
df_comb = pd.concat([df_h_temp, df_a_temp], ignore_index=True)
df_comb = df_comb.sort_values("source").drop_duplicates("post_id", keep="last")
label_sets = df_comb[["post_id", "label"]]

# join to main
df_combined = df_main[df_main["post_id"].isin(label_sets["post_id"])].merge(
    label_sets, on="post_id", how="inner"
)
print(f"  - Supervised samples: {len(df_combined):,}")

[3/6] Creating labels...
  - Supervised samples: 1,006


In [5]:
# STEP 4: Build features (TF-IDF + NMF + metadata scaler)
print("[4/6] Building features...")

tokens = df_combined[TEXT_COL].fillna("").map(tokenize)

# TF-IDF
X_tfidf = vec.transform(tokens)

# NMF topics
W_topics = nmf.transform(X_tfidf)
X_nmf = csr_matrix(W_topics)

# Metadata
doc_len = np.array([len(t) for t in tokens], dtype=float)[:, None]
has_url = (
    df_combined[TEXT_COL]
    .fillna("")
    .str.contains("http", case=False)
    .astype(int)
    .values[:, None]
).astype(float)
nrc = (
    df_combined.get("anxiety_score", pd.Series(0, index=df_combined.index))
    .fillna(0)
    .values[:, None]
).astype(float)
meta = np.hstack([np.log1p(doc_len), has_url, nrc])
meta_scaled = meta_scaler.transform(meta)
X_meta = csr_matrix(meta_scaled)

# Combine all
from scipy.sparse import hstack

X = hstack([X_tfidf, X_nmf, X_meta], format="csr")
X.data = np.nan_to_num(X.data, nan=0.0)
y_true = df_combined["label"].values
texts = df_combined[TEXT_COL].fillna("").tolist()

print(f"  - Feature shape: {X.shape}")
print(f"  - Total samples: {len(y_true)}")
print(f"  - Positive: {y_true.sum()} ({y_true.sum()/len(y_true)*100:.1f}%)")

[4/6] Building features...


/Users/mariamckay/code/umich/milestone2/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


  - Feature shape: (1006, 9717)
  - Total samples: 1006
  - Positive: 173 (17.2%)


In [6]:
# STEP 5: Predict
print("[5/6] Making predictions...")
probs = calib_model.predict_proba(X)[:, 1]
preds = (probs >= threshold).astype(int)

fp_mask = (y_true == 0) & (preds == 1)
fn_mask = (y_true == 1) & (preds == 0)
fp_idx = np.where(fp_mask)[0]
fn_idx = np.where(fn_mask)[0]

print("\n[6/6] FAILURE ANALYSIS")
print("=" * 70)
print(f"Threshold: {threshold:.6f}")
print(f"Total samples: {len(y_true)}")
print(f"False Positives: {len(fp_idx)}")
print(f"False Negatives: {len(fn_idx)}")

[5/6] Making predictions...

[6/6] FAILURE ANALYSIS
Threshold: 0.250000
Total samples: 1006
False Positives: 120
False Negatives: 16


In [7]:
# STEP 6: Inspect top FP and FN with token contributions
# Try to extract underlying LR coef for TF-IDF feature contributions
coef = None
try:
    base_clf = calib_model.calibrated_classifiers_[0].estimator
    coef = (
        base_clf.coef_[0]
        if hasattr(base_clf.coef_, "shape") and base_clf.coef_.ndim == 2
        else base_clf.coef_
    )
except Exception:
    print("[WARNING] Could not extract coefficients for token analysis")

# Best FP
if len(fp_idx) > 0:
    best_fp = fp_idx[np.argmax(probs[fp_idx])]
    print("\n" + "-" * 70)
    print("HIGHEST CONFIDENCE FALSE POSITIVE (FP)")
    print("-" * 70)
    print(f"Index: {best_fp}")
    print("True label: 0 (not anxiety)")
    print("Predicted: 1 (anxiety)")
    print(f"Probability: {probs[best_fp]:.4f}")
    print("\nText excerpt (first 200 chars):")
    print(texts[best_fp][:200] + "...")

    if coef is not None:
        try:
            x_tfidf = X_tfidf[best_fp]
            contributions = x_tfidf.multiply(coef[: X_tfidf.shape[1]]).toarray()[0]
            feature_names = vec.get_feature_names_out()
            top_pos_idx = np.argsort(-contributions)[:8]
            print("\nTop anxiety-indicating tokens:")
            for idx in top_pos_idx:
                if contributions[idx] > 0:
                    print(f"  '{feature_names[idx]}': +{contributions[idx]:.4f}")
        except Exception as e:
            print(f"[WARNING] Could not extract token contributions: {e}")

# Best FN
if len(fn_idx) > 0:
    best_fn = fn_idx[np.argmin(probs[fn_idx])]
    print("\n" + "-" * 70)
    print("HIGHEST CONFIDENCE FALSE NEGATIVE (FN)")
    print("-" * 70)
    print(f"Index: {best_fn}")
    print("True label: 1 (anxiety)")
    print("Predicted: 0 (not anxiety)")
    print(f"Probability: {probs[best_fn]:.4f}")
    print("\nText excerpt (first 200 chars):")
    print(texts[best_fn][:200] + "...")

    if coef is not None:
        try:
            x_tfidf = X_tfidf[best_fn]
            contributions = x_tfidf.multiply(coef[: X_tfidf.shape[1]]).toarray()[0]
            feature_names = vec.get_feature_names_out()
            top_pos_idx = np.argsort(-contributions)[:8]
            top_neg_idx = np.argsort(contributions)[:8]
            print("\nTop anxiety-indicating tokens:")
            for idx in top_pos_idx:
                if contributions[idx] > 0:
                    print(f"  '{feature_names[idx]}': +{contributions[idx]:.4f}")
            print("\nTop non-anxiety-indicating tokens:")
            for idx in top_neg_idx:
                if contributions[idx] < 0:
                    print(f"  '{feature_names[idx]}': {contributions[idx]:.4f}")
        except Exception as e:
            print(f"[WARNING] Could not extract token contributions: {e}")

print("\n" + "=" * 70 + "\n")


----------------------------------------------------------------------
HIGHEST CONFIDENCE FALSE POSITIVE (FP)
----------------------------------------------------------------------
Index: 410
True label: 0 (not anxiety)
Predicted: 1 (anxiety)
Probability: 1.0000

Text excerpt (first 200 chars):
idk how much more of this life i can take

i know this will probably seem just like useless ranting but i have no one else to talk to about this. i 29f am a sahm to 5 kids. up until a few years ago th...

Top anxiety-indicating tokens:
  'you some': +0.0583
  'hard': +0.0524
  'contact': +0.0462
  'sorry': +0.0414
  'afford': +0.0395
  'what': +0.0380
  'few': +0.0356
  'that you': +0.0352

----------------------------------------------------------------------
HIGHEST CONFIDENCE FALSE NEGATIVE (FN)
----------------------------------------------------------------------
Index: 654
True label: 1 (anxiety)
Predicted: 0 (not anxiety)
Probability: 0.0455

Text excerpt (first 200 chars):
religion mega